In [267]:
import os
import numpy as np
from PIL import Image

dataset = []
labels = []
for filename in os.listdir("dataset"):
    if filename.endswith(".jpg"):
        img_path = os.path.join("dataset", filename)
        im = Image.open(img_path)
        dataset.append(np.asarray(im, dtype=np.float32))
        labels.append(filename.replace(".jpg", ""))

In [268]:
import pandas as pd
import numpy as np
import cv2
import glob
import imutils
from imutils import paths
import os
import os.path

captcha_image = 'dataset/1rb5rb.jpg'
filename = os.path.basename(captcha_image)
captcha_text = os.path.splitext(filename)[0]


image = cv2.imread(captcha_image)


def first_element(val):
    return val[0]

def merge_boxes(b1, b2):
    b1_xa, b1_ya, b1_w, b1_h = b1[0], b1[1], b1[2], b1[3]
    b2_xa, b2_ya, b2_w, b2_h = b2[0], b2[1], b2[2], b2[3]
    b1_xb, b1_yb = b1_xa + b1_w, b1_ya + b1_h
    b2_xb, b2_yb = b2_xa + b2_w, b2_ya + b2_h
    
    xa = min(b1_xa, b2_xa)
    ya = min(b1_ya, b2_ya)
    xb = max(b1_xb, b2_xb)
    yb = max(b1_yb, b2_yb)
    w = xb - xa
    h = yb - ya
    return [xa, ya, w, h]
    
def merge_bounding_boxes(bounding_boxes):
    i = 1
    res_boxes = []
    skip_box = False
    for box in bounding_boxes[1:]:
        prev_box = bounding_boxes[i - 1]
        if (prev_box[0] == box[0]) or (prev_box[0] + prev_box[2]) == (box[0] + box[2]):
            res_boxes.append(merge_boxes(prev_box, box))
            skip_box = True
        else:
            if not skip_box:
                res_boxes.append(prev_box)
            skip_box = False
        i = i + 1
    if not skip_box:
        res_boxes.append(bounding_boxes[i - 1])
    return res_boxes

def split_large_bounding_boxes(bounding_boxes):
    res = []
    for box in bounding_boxes:
        if box[2] > 23:
            b1 = box.copy()
            b2 = box.copy()
            b1[2] = int(b1[2] / 2)
            b2[2] = int(b2[2] / 2)
            res.append(b1)
            res.append(b2)
        else:
            res.append(box)
    return res

def padded(image, target_w, target_h):
    h, w = image.shape[:2]
    if w < target_w or h < target_h:
        extra_w = int((target_w - w)/2)
        extra_w_1 = target_w - w - extra_w
        extra_h = int((target_h - h)/2)
        extra_h_1 = target_h - h - extra_h
        padded = cv2.copyMakeBorder(image, extra_h, extra_h_1, extra_w, extra_w_1, cv2.BORDER_CONSTANT, value=[255,255,255])        
        return padded
    else:
        return image

def split_image(image):
    global max_h, max_w
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_border = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_CONSTANT, value=[0,0,0])
    binarized = cv2.threshold(gray_border, 50, 255, cv2.THRESH_BINARY_INV, cv2.THRESH_OTSU)[1]
    contours, hierarchies = cv2.findContours(binarized, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    index = 0 
    letters = []
    bounding_boxes = []
    for contour in contours:
        hr = hierarchies[0][index]
        x, y, w, h = cv2.boundingRect(contour)

        if (hr[3] == 0):
            bounding_boxes.append([x, y, w, h])    
        index = index +1
        
    bounding_boxes.sort(key=first_element)        
    bounding_boxes = merge_bounding_boxes(bounding_boxes)
    bounding_boxes = split_large_bounding_boxes(bounding_boxes)
    if len(bounding_boxes) > 6:
        print(bounding_boxes)
    
    for box in bounding_boxes:
        x, y, w, h = box[0], box[1], box[2], box[3]
        crop_img = binarized[y:y+h, x:x+w]
        letters.append(padded(crop_img, 28, 28))
            
    return letters


n = 0
for filename in os.listdir("dataset"):
    if filename.endswith(".jpg"):
        img_path = os.path.join("dataset", filename)
        label = filename.replace(".jpg", "")
        image = cv2.imread(img_path)
        letters = split_image(image)
        if len(letters) != 6 or len(label) != 6:
            print(label)
            display(Image.fromarray(image))
            print("!")
            print(len(letters))
            for pic in letters:
                display(Image.fromarray(pic))
                print(pic.shape)
        else:
            pi = 0
            for pic in letters:
                c = label[pi]
                cv2.imwrite("secondary_dataset/%s-%04d.jpg" % (c, n), pic)
                display(Image.fromarray(pic))
                n = n + 1
                pi = pi + 1



[[10, 8, 17, 21], [23, 13, 20, 21], [28, 8, 6, 5], [37, 8, 6, 5], [46, 8, 14, 21], [61, 13, 23, 16], [80, 8, 11, 26]]
9jj2wj


!
7


(28, 28)


(28, 28)


(28, 28)


(28, 28)


(28, 28)


(28, 28)


(28, 28)
